In [ ]:
#IMPORTING LIBRARIES.
import pandas as pd 
import numpy as np 
import matplotlib.pyplot as plt
import seaborn as sns

In [ ]:
df=pd.read_csv("/kaggle/input/playground-series-s3e15/data.csv")
df["original"]=0
df.head()

In [ ]:
df1=pd.read_csv("/kaggle/input/predicting-heat-flux/Data_CHF_Zhao_2020_ATE.csv")
df1["original"]=1
df1.head()

In [ ]:
df1.isna().sum()

In [ ]:
df["author"].unique()

In [ ]:
df1["author"].unique()

In [ ]:
df1["geometry"].unique()

In [ ]:
df["geometry"].unique()

In [ ]:
#how to check the min max values in each column of dataframe 

# for i in df.columns:
    # print('columns {i} has',df[i].min() , df[i].max()).format

In [ ]:
concat_df=pd.concat([df,df1],axis=0);
concat_df

In [ ]:
from sklearn.preprocessing import LabelEncoder
le=LabelEncoder()

concat_df["author"]=le.fit_transform(concat_df["author"])
concat_df["geometry"]=le.fit_transform(concat_df["geometry"])

In [ ]:
concat_df["author"]=concat_df["author"].replace({10:np.nan})
concat_df["geometry"]=concat_df["geometry"].replace({2:np.nan})
concat_df

In [ ]:
df_=concat_df.drop(columns=["x_e_out [-]","id"])
df_

In [ ]:
# using sklearn knn imputer

from sklearn.impute import KNNImputer

imputer = KNNImputer(n_neighbors=4)  # Set the number of neighbors

# Fit the imputer on the data
imputer.fit(df_)

# Transform the data by imputing missing values
X_imputed = imputer.transform(df_)

In [ ]:
# import pandas as pd
# from impyute.imputation.cs import mice

In [ ]:
# # Load your dataset with missing values
# data = df_

# # Perform MICE imputation
# imputed_data = mice(data.values)

# # Convert the imputed data array back to a DataFrame
# imputed_df = pd.DataFrame(imputed_data, columns=data.columns)

In [ ]:
# imputed_df

In [ ]:
# Verify if any missing values remain
# print(imputed_df.isnull().sum())

In [ ]:
# Convert the imputed data back to a DataFrame
imputed_df = pd.DataFrame(X_imputed, columns=df_.columns)

# Print the imputed DataFrame
imputed_df

In [ ]:
imputed_df.isna().sum().sum()

In [ ]:
y=concat_df["x_e_out [-]"]
y

In [ ]:
y=pd.DataFrame(y).reset_index()
y

In [ ]:
new_df=pd.concat([imputed_df,y],axis=1)

In [ ]:
new_df=new_df.drop(columns=["index"])
new_df

In [ ]:
# Find rows with null values in the specified column
null_rows = new_df[new_df["x_e_out [-]"].isnull()]

# Print the null rows
null_rows

In [ ]:
# Find rows with non-null values in the specified column
not_null_rows = new_df[new_df["x_e_out [-]"].notnull()]

# Print the non-null rows
not_null_rows

In [ ]:
not_null_rows=not_null_rows[not_null_rows["original"]==0]

In [ ]:
# not_null_rows=not_null_rows[:-len(df1)]

In [ ]:
y=not_null_rows["x_e_out [-]"]
X=not_null_rows.drop(columns=["x_e_out [-]","original","author","geometry"])

In [ ]:
X.columns = [col.replace('[', '').replace(']', '').replace('<', '').replace('>', '') for col in X.columns]

In [ ]:
from sklearn.metrics import mean_squared_error
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor, BaggingRegressor, AdaBoostRegressor, ExtraTreesRegressor
from sklearn.neural_network import MLPRegressor
from sklearn.neighbors import KNeighborsRegressor
from sklearn.linear_model import LinearRegression, RidgeCV, Ridge, SGDRegressor
from lightgbm import LGBMRegressor
from xgboost import XGBRegressor
from catboost import CatBoostRegressor
from sklearn.model_selection import KFold
import numpy as np
import pandas as pd

In [ ]:
def regress_models(x, X, y):
    kf = KFold(n_splits=5, shuffle=True, random_state=42)
    rmse_scores = []
    for train_index, test_index in kf.split(X,y):
        X_train, X_test = X.iloc[train_index], X.iloc[test_index]
        y_train, y_test = y.iloc[train_index], y.iloc[test_index]
        x.fit(X_train, y_train)
        y_pred = x.predict(X_test)
        rmse_scores.append(np.sqrt(mean_squared_error(y_test, y_pred)))
    model_name = x.__class__.__name__
    return model_name, round(np.mean(rmse_scores), 3)

algo = [DecisionTreeRegressor(),
        RandomForestRegressor(),
        GradientBoostingRegressor(),
        BaggingRegressor(),
        AdaBoostRegressor(),
        ExtraTreesRegressor(),
        MLPRegressor(),
        KNeighborsRegressor(),
        LinearRegression(),
        CatBoostRegressor(),
        XGBRegressor(),
        LGBMRegressor(),
        # tpot.fitted_pipeline_,
        SGDRegressor(loss='squared_loss', alpha=0.0001, max_iter=1000, tol=1e-3),
        RidgeCV(alphas=[1e-3, 1e-2, 1e-1, 1], cv=5),
        Ridge(alpha=1.0)]

score = []
for a in algo:
    score.append(regress_models(a, X, y))

pd.DataFrame(score, columns=['Model', 'RMSE'])

In [ ]:
cat_params = {
    'objective':'RMSE',
    "loss_function": "RMSE",
    'eval_metric': "RMSE",
    "n_estimators": 5000,
    "verbose" : 1000,
    #"learning_rate": 0.07,
    "random_state": 45,
    #"l2_leaf_reg": 3.0,
    #"bootstrap_type": "Bayesian",
    #"bagging_temperature": 1.0,
    #"subsample": 1.0,
    #"random_strength": 1.0,
    #"depth": 6,
    #"grow_policy": "Lossguide",
    #"grow_policy": "Depthwise",
    #"max_leaves": 41,
    #"od_type": "Iter",
    #"od_wait": 20,
    #"border_count": 254,
}

In [ ]:
from catboost import CatBoostRegressor
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import KFold
import numpy as np

n_splits = 10
rmse_scores = []
best_iterations = []
oof_predictions = np.zeros(len(X))  # Array to store the OOF predictions

kf = KFold(n_splits=n_splits, random_state=42, shuffle=True)

for fold, (train_index, val_index) in enumerate(kf.split(X, y)):
    X_train, X_val = X.iloc[train_index], X.iloc[val_index]
    y_train, y_val = y.iloc[train_index], y.iloc[val_index]
    
    model = CatBoostRegressor(**cat_params)
    model.fit(X_train, y_train, eval_set=(X_val, y_val), early_stopping_rounds=500, verbose=False)
    
    best_iteration = model.best_iteration_
    best_iterations.append(best_iteration)
    
    y_pred = model.predict(X_val)
    oof_predictions[val_index] = y_pred  # Store the OOF predictions
    
    score = mean_squared_error(y_val, y_pred, squared=False)
    rmse_scores.append(score)
    print(f"Fold {fold}: Validation RMSE = {score}")
    
mean_rmse_score = np.mean(rmse_scores)
print(f"Mean RMSE = {mean_rmse_score}")

mean_best_iteration = int(np.mean(best_iterations))
print(f"Mean Best Iteration = {mean_best_iteration}")

# Calculate OOF RMSE
oof_rmse = mean_squared_error(y, oof_predictions, squared=False)
print(f"OOF RMSE = {oof_rmse}")

In [ ]:
cat_params = {
    'objective':'RMSE',
    "loss_function": "RMSE",
    'eval_metric': "RMSE",
    "n_estimators": mean_best_iteration,
    "verbose" : 1528,
    #"learning_rate": 0.07,
    "random_state": 45,
}

In [ ]:
model=CatBoostRegressor(**cat_params)
# CatBoostRegressor(**cat_params)
# model.fit(X, y)

In [ ]:
test=null_rows.drop(columns=["x_e_out [-]","original","author","geometry"])
                    
test

test.columns = [col.replace('[', '').replace(']', '').replace('<', '').replace('>', '') for col in test.columns]

In [ ]:
model.fit(X,y)

y_pred=model.predict(test)
y_pred

In [ ]:
sub=pd.read_csv("/kaggle/input/playground-series-s3e15/sample_submission.csv")
sub

In [ ]:
sub=sub.set_index("id")
sub

In [ ]:
sub["x_e_out [-]"]=y_pred
sub

In [ ]:
sub.to_csv("submission.csv")